# **GAN FED** Protein homo

## Required libraries and configuration

In [1]:
!pip install -q flwr[simulation]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 43.3 MB/s eta 0:00:00



Import required libraries

In [2]:
import os

import re
import time
import random
import sys

import time

from typing import Dict, Optional, Tuple, List
from collections import OrderedDict

import flwr as fl
from flwr.common import Metrics

import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow.keras import models, layers, losses, metrics, optimizers

from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import SMOTE, RandomOverSampler
from collections import Counter

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics as skmetrics
from sklearn.metrics import confusion_matrix

In [4]:
from GAN import Gan

Define some parameters for the simulation, such as the number of clients in the federated scenario, the number of federated rounds, the number of epochs of each client before communicating, and the batch size for training phase

In [5]:
# Some parameters
NUM_CLIENTS = 10 # Number of clients in the federated scenario
NUM_ROUNDS = 10 #5 Number of learning rounds in the federated computation
NUM_EPOCHS = 5 #3 Number of epochs that the local dataset is seen each round
BATCH_SIZE = 20 # Batch size for training phase

# Define the seed for random numbers
seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)
random.seed(seed)

## Loading and preparing the input data

The Sentiment140 dataset is not available in TFF, so it needs to be loaded from other source such as the tfds (tensorflow datasets) library. Then, it is adapted to the TFF format, so it can be used to train a model using TFF.

Note: We download the full Sentiment140 dataset, but in order to be able to execute the experiments in reasonable time, we are going to use a portion of it. In this notebook we are selecting just 1% of the data for training and 10% for testing purposes.

In [6]:
dset_name = 'protein_homo'
dset = fetch_datasets()[dset_name]

In [7]:
X = pd.DataFrame(StandardScaler().fit_transform(dset.data))
y = dset.target

y[y == -1] = 0
X['label'] = y

dtrain, dtest = train_test_split(X, test_size=0.2, random_state=42, stratify=y)

In [8]:
dtest = dtest.iloc[:-1,:]

In [9]:
dtrain.shape

(116600, 75)

In [10]:
dtrain['label'].value_counts()

0    115563
1      1037
Name: label, dtype: int64

Select the text and polarity columns from the original data, and transform it to a dataframe so it can be later used in TFF.

At this point, we do not select the 'user' column because we will create random IID partitions, so the 'user' column is not neccesary. If the user want to try it with non-IID partition, he/she may also keep the 'user' column and use it as client identifier instead of the following random user ID.

In [11]:
# Create a random list of ids. Each instance is given a random id, indicating the client where will be distributed
ids_train = [i for i in range(NUM_CLIENTS) for _ in range(len(dtrain)//NUM_CLIENTS)]
random.Random(seed).shuffle(ids_train)

# Add the id assignment to the dataframe
dtrain['user'] = ids_train
unique_ids_train = list(set(ids_train))

In [12]:
# Do the same with the test data
ids_test = [i for i in range(NUM_CLIENTS) for _ in range(len(dtest)//NUM_CLIENTS)]
random.Random(seed+1).shuffle(ids_test)
dtest['user'] = ids_test
unique_ids_test = list(set(ids_test))

For the sake of simplicity, in this notebook we will be dealing with a binary problem. For that purpose, we remove the neutral tweets, so the classifier's aim is to differentiate between positive and negative tweets.

Create the DataLoaders for each client. It creates an array of datasets, one for each client.

In [13]:
# Create DataLoaders for each client
train_data = []
test_data = []
for i in range(NUM_CLIENTS):
    train_data.append(dtrain.loc[dtrain['user']==unique_ids_train[i]])
    test_data.append(dtest.loc[dtest['user']==unique_ids_test[i]])

## Create a Deep Learning model

In [14]:
def create_keras_model_gan():
    model = tf.keras.Sequential(name="Generator_model")
    model.add(tf.keras.layers.Dense(15, activation='relu',
                                    kernel_initializer='he_uniform',
                                    input_dim=74))
    model.add(tf.keras.layers.Dense(30, activation='relu'))
    model.add(tf.keras.layers.Dense(74, activation='linear'))
    return model


In [15]:
# Class-independent methods to get and set the parameters of a model
def get_parameters(model):
    return model.get_weights()

def set_parameters(model, parameters):
    model.set_weights(parameters)

In [16]:
class FlowerClientGAN(fl.client.NumPyClient):
    def __init__(self, model, client_train_data, client_test_data):
        # Init the client setting the x (text) and the y (polarity) for both train and testing
        self.model = model
        #x_train, y_train = client_train_data.iloc[:, :-1], client_train_data.iloc[:, -1]
        #x_test, y_test = client_test_data.iloc[:, :-1], client_test_data.iloc[:, -1]
        client_train_data = client_train_data.drop('user', axis=1)
        client_test_data = client_test_data.drop('user', axis=1)
        # print(f"Client_train_data: {client_train_data.info()}")
        x_train, y_train = client_train_data.drop('label',axis=1), client_train_data['label'].to_numpy()
        x_test, y_test = client_test_data.drop('label',axis=1), client_test_data['label'].to_numpy()
        # print(f"X_train: {x_train}")
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        # self.train_dataset = tf.data.Dataset.from_tensor_slices((dict(x_train), y_train))
        # self.train_dataset = self.train_dataset.batch(BATCH_SIZE)        

    def get_properties(self, config):
        """Get properties of client."""
        raise Exception("Not implemented")

    def get_parameters(self, config):
        """Get parameters of the local model."""
        return get_parameters(self.model)

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""
        # Update local model parameters
        set_parameters(self.model, parameters)

        # Get hyperparameters for this round
        batch_size: int = BATCH_SIZE
        epochs: int = NUM_EPOCHS
            
        X_train_pos = self.x_train[self.y_train == 1]

        model = Gan(data=X_train_pos)
        generator = self.model
        discriminator = model._discriminator()
        gan_model = model._GAN(generator=generator, discriminator=discriminator)
        
        trained_model = model.train(generator, discriminator, gan_model, verbose=False)

        # Return updated model parameters and results
        parameters_prime = get_parameters(model=trained_model)
        num_examples_train = len(self.x_train)

        #results = {"loss": history.history["loss"][0]}
        results = {"loss": 0.5}

        return parameters_prime, num_examples_train, results

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""

        # Update local model with global parameters
        set_parameters(self.model, parameters)

        # Get config values
        steps: int = 10

        # Evaluate global model parameters on the local test data and return results
        #loss, accuracy = self.model.evaluate(self.x_test)
        #loss = self.model.evaluate(self.x_test, self.y_test, 1)
        loss = 0.01
        num_examples_test = len(self.x_test)
        
        self.model.save_weights('gan_fl%f.h5' % time.time())

        return loss, num_examples_test, {"accuracy": 0.95} # tpr*tnr}

In [17]:
def client_fn_gan(cid: str) -> FlowerClientGAN:
    # Load model
    model = create_keras_model_gan()

    # Note: each client gets a different train/test data
    client_train_data = train_data[int(cid)]
    client_test_data = test_data[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClientGAN(model, client_train_data, client_test_data)

In [18]:
# Define the configuration during training/fitting the model. It is used later when defining the FedAvg strategy.
def fit_config(server_round: int):
    config = {
        "batch_size": BATCH_SIZE,
        "local_epochs": NUM_EPOCHS
    }
    return config

# Define the configuration during evaluation. It is used later when defining the FedAvg strategy.
def evaluate_config(server_round: int):
    return {"val_steps": 3}

In [19]:
# Define strategy
strategy = fl.server.strategy.FedAvg(
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    #evaluate_metrics_aggregation_fn=weighted_average,
    #fraction_evaluate=0.1,  # Sample 10% of available clients for evaluation
)

# Start simulation
fl_sim = fl.simulation.start_simulation(
    client_fn=client_fn_gan,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-08-19 17:37:47,746 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-08-19 17:37:49,648	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-08-19 17:37:50,780 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 16.0, 'node:192.168.1.129': 1.0, 'object_store_memory': 8438874931.0, 'memory': 16877749863.0}
INFO flwr 2023-08-19 17:37:50,781 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-19 17:37:50,782 | server.py:273 | Requesting initial parameters from one random client
(pid=2934468) 2023-08-19 17:37:51.021122: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=2934468) 2023-08-19 17:37:51.021150: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your 

DEBUG flwr 2023-08-19 17:39:05,530 | server.py:218 | fit_round 4: strategy sampled 10 clients (out of 10)
(launch_and_fit pid=2934474) 2023-08-19 17:39:08.300530: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled
(launch_and_fit pid=2934465) 2023-08-19 17:39:08.403605: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled
(launch_and_fit pid=2934468) 2023-08-19 17:39:13.930668: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled [repeated 6x across cluster]
(launch_and_fit pid=2934476) 2023-08-19 17:39:19.996237: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled [repeated 8x across cluster]
(launch_and_fit pid=2934472) 2023-08-19 17:39:25.952315: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled [repeated 8x across cluste

(launch_and_fit pid=2934468) 2023-08-19 17:41:31.296194: W tensorflow/core/data/root_dataset.cc:200] Optimization loop failed: CANCELLED: Operation was cancelled [repeated 10x across cluster]
DEBUG flwr 2023-08-19 17:41:44,145 | server.py:232 | fit_round 10 received 10 results and 0 failures
DEBUG flwr 2023-08-19 17:41:44,158 | server.py:168 | evaluate_round 10: strategy sampled 10 clients (out of 10)
DEBUG flwr 2023-08-19 17:41:44,462 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-08-19 17:41:44,463 | server.py:147 | FL finished in 232.17307465299928
INFO flwr 2023-08-19 17:41:44,464 | app.py:218 | app_fit: losses_distributed [(1, 0.01), (2, 0.01), (3, 0.01), (4, 0.01), (5, 0.01), (6, 0.01), (7, 0.01), (8, 0.01), (9, 0.01), (10, 0.01)]
INFO flwr 2023-08-19 17:41:44,465 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-19 17:41:44,466 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-08-19 17:41:44,466 | app.py:221 |

In this case we use a model composed by a pre-trained model from tfhub, as well as dense layers. The pre-trained model is not updated in the example; however, the ``trainable`` parameter can be set to ``True``, so such layers are also fine-tuned in the collaborative training.

Note that any network architecture supported by keras can be used. Besides, Flower also supports PyTorch models, but in contrast to Use Case 1, we use a TFF model here to provide a variety of options.

In [23]:
def create_keras_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(256, activation='relu', input_shape=[74]))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1))

    # Compile the model
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer=tf.optimizers.Adam(),
        metrics=[tf.metrics.AUC(from_logits=True)]
    )

    return model

## Training in the federated scenario

First, we create a FlowerClient class, that includes the information of each simulated client. The class has three methods:
 * `get_parameters`: Get the parameters of the model to send them to the server
 * `fit`: Reveives the model parameters from the server, trains it with local data, and return the updated model parameters to the server
 * `evaluate`: Receives the model from the server and evaluates it with local data

In [24]:
def _df(data):
    df = pd.DataFrame(data)
    for c in range(df.shape[1]):
        mapping = {df.columns[c]: c}
        df = df.rename(columns=mapping)
    return df

In [25]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [26]:
# Class-independent methods to get and set the parameters of a model
def get_parameters(model):
        return model.get_weights()

def set_parameters(model, parameters):
    model.set_weights(parameters)

In [27]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, client_train_data, client_test_data):
        # Init the client setting the x (text) and the y (polarity) for both train and testing
        self.model = model

        client_train_data = client_train_data.drop('user', axis=1)
        client_test_data = client_test_data.drop('user', axis=1)

        x_train, y_train = client_train_data.drop('label',axis=1), client_train_data['label'].to_numpy()
        x_test, y_test = client_test_data.drop('label',axis=1), client_test_data['label'].to_numpy()

        # GAN
        X_train_pos = x_train[y_train == 1]
        
        model_gan = Gan(data=X_train_pos)
        generator = model_gan._generator()
        generator.load_weights('gan_fl1691763281.591241.h5')
        discriminator = model_gan._discriminator()
        gan_model = model_gan._GAN(generator=generator, discriminator=discriminator)
        trained_model = model_gan.train(generator, discriminator, gan_model, verbose=False)
        
        # Samples to generate
        sampling_strategy = .7
        num_examples = int((Counter(y_train)[0] - Counter(y_train)[1]) * sampling_strategy)
        noise = np.random.normal(0, 1, (X_train_pos.shape[0] + num_examples, X_train_pos.shape[1]))
        new_data = _df(data=trained_model.predict(noise))

        X_res = np.concatenate((x_train, new_data))
        y_res = np.concatenate((y_train, np.ones(new_data.shape[0])))

        # RUS
        X_eq, y_eq = RUS(X_res, y_res, frac=.02)

        self.x_train, self.y_train = X_eq, y_eq
        self.x_test, self.y_test = x_test, y_test

    def get_properties(self, config):
        """Get properties of client."""
        raise Exception("Not implemented")

    def get_parameters(self, config):
        """Get parameters of the local model."""
        return get_parameters(self.model)

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""
        # Update local model parameters
        set_parameters(self.model, parameters)

        # Get hyperparameters for this round
        batch_size: int = BATCH_SIZE
        epochs: int = NUM_EPOCHS

        # Train the model using hyperparameters from config
        history = self.model.fit(
            self.x_train,
            self.y_train,
            batch_size,
            epochs,
            verbose=0
        )

        # Return updated model parameters and results
        parameters_prime = get_parameters(model=self.model)
        num_examples_train = len(self.x_train)
        results = {
            "loss": history.history["loss"][0],
            #"accuracy": history.history["accuracy"][0],

        }
        return parameters_prime, num_examples_train, results

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""

        # Update local model with global parameters
        set_parameters(self.model, parameters)

        # Get config values
        steps: int = 10

        # Evaluate global model parameters on the local test data and return results
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, 1)
        num_examples_test = len(self.x_test)

        y_pred = self.model.predict(self.x_test)
        y_pred = [1 if pred >= 0.5 else 0 for pred in y_pred]

        cm = confusion_matrix(self.y_test, y_pred)
        tn = cm[0][0]
        fn = cm[1][0]
        tp = cm[1][1]
        fp = cm[0][1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)

        cr = skmetrics.classification_report(self.y_test, y_pred)
        print(cr)

        return loss, num_examples_test, {"TPR*TNR": tpr*tnr}

To simulate the federated scenario in a single machine, the client_fn method allows to create FlowerClients on demand, given the client id.

Note that each client is passed both training and testing local data, so the evaluation over test data is done during the simulation itself.

In [28]:
def client_fn(cid: str) -> FlowerClient:
    # Load model
    model = create_keras_model()

    # Note: each client gets a different train/test data
    client_train_data = train_data[int(cid)]
    client_test_data = test_data[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(model, client_train_data, client_test_data)

In order to show averaged evaluations metrics beyond loss, we should define a method to do that; in this case, the accuracy is weighted averaged.

In [29]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["TPR*TNR"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"TPR*TNR": sum(accuracies) / sum(examples)}

Define a training strategy with the weighted FedAvg algorithm.

Then, start the simulation indicating the method to create clients, the number of clients in the simulation, the number of rounds, and the strategy (i.e., the FedAvg strategy to combine local updates). The simulation covers both the federated model training as well as evaluating the model with each local test data.

In [30]:
# Define the configuration during training/fitting the model. It is used later when defining the FedAvg strategy.
def fit_config(server_round: int):
    config = {
        "batch_size": BATCH_SIZE,
        "local_epochs": NUM_EPOCHS
    }
    return config

# Define the configuration during evaluation. It is used later when defining the FedAvg strategy.
def evaluate_config(server_round: int):
    return {"val_steps": 5}

In [31]:
# Define strategy
strategy = fl.server.strategy.FedAvg(
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl_sim = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-08-11 20:08:57,115 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


(launch_and_fit pid=3525) >200, d1=0.183, d2=1.876 [repeated 287x across cluster]


2023-08-11 20:09:02,557	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-08-11 20:09:03,933 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 22849349223.0, 'object_store_memory': 11424674611.0, 'node:127.0.1.1': 1.0, 'CPU': 16.0}
INFO flwr 2023-08-11 20:09:03,934 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-11 20:09:03,935 | server.py:273 | Requesting initial parameters from one random client
(pid=113729) 2023-08-11 20:09:04.180477: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
(pid=113729) 2023-08-11 20:09:04.180506: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(launch_and_get_parameters pid=113729) 2023-08-11 20:09:05.547340: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

1721/2915 [================>.............] - ETA: 2s - loss: 0.0631 - auc_1: 0.9321


 855/2915 [=======>......................] - ETA: 4s - loss: 0.0364 - auc_1: 0.9976 [repeated 2x across cluster]
(launch_and_evaluate pid=113727)               precision    recall  f1-score   support
(launch_and_evaluate pid=113727) 
(launch_and_evaluate pid=113727)            0       1.00      0.98      0.99      2892
(launch_and_evaluate pid=113727)            1       0.31      0.87      0.46        23
(launch_and_evaluate pid=113727) 
(launch_and_evaluate pid=113727)     accuracy                           0.98      2915
(launch_and_evaluate pid=113727)    macro avg       0.66      0.93      0.73      2915
(launch_and_evaluate pid=113727) weighted avg       0.99      0.98      0.99      2915
(launch_and_evaluate pid=113727) 
 869/2915 [=======>......................] - ETA: 4s - loss: 0.0462 - auc_1: 0.9975 [repeated 2x across cluster]


...

1680/2915 [================>.............] - ETA: 2s - loss: 0.1063 - auc_19: 0.9678


2915/2915 [==============================] - 7s 2ms/step - loss: 0.1330 - auc_19: 0.9727


2835/2915 [============================>.] - ETA: 0s - loss: 0.1555 - auc_20: 0.9542
(launch_and_evaluate pid=113732)               precision    recall  f1-score   support
(launch_and_evaluate pid=113732) 
(launch_and_evaluate pid=113732)            0       1.00      0.98      0.99      2893
(launch_and_evaluate pid=113732)            1       0.22      0.91      0.35        22
(launch_and_evaluate pid=113732) 
(launch_and_evaluate pid=113732)     accuracy                           0.97      2915
(launch_and_evaluate pid=113732)    macro avg       0.61      0.94      0.67      2915
(launch_and_evaluate pid=113732) weighted avg       0.99      0.97      0.98      2915
(launch_and_evaluate pid=113732) 
 875/2915 [========>.....................] - ETA: 4s - loss: 0.1036 - auc_19: 0.9954 [repeated 2x across cluster]


DEBUG flwr 2023-08-11 20:20:28,934 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-08-11 20:20:28,935 | server.py:147 | FL finished in 671.055378428
INFO flwr 2023-08-11 20:20:28,936 | app.py:218 | app_fit: losses_distributed [(1, 0.05392354167997837), (2, 0.10520271733403205), (3, 0.1063789702951908), (4, 0.11067049130797386), (5, 0.10285662561655044), (6, 0.12072182446718216), (7, 0.10957779437303543), (8, 0.11824006587266922), (9, 0.12832544967532158), (10, 0.1276228755712509)]
INFO flwr 2023-08-11 20:20:28,937 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-08-11 20:20:28,938 | app.py:220 | app_fit: metrics_distributed {'TPR*TNR': [(1, 0.882634710146813), (2, 0.8970806566225734), (3, 0.8978881872419666), (4, 0.9088566066696251), (5, 0.911663094082676), (6, 0.9160532709168877), (7, 0.9141170593754905), (8, 0.9202585911239172), (9, 0.9082675729549773), (10, 0.9058414615466825)]}
INFO flwr 2023-08-11 20:20:28,939 | app.py:221 | a

1108/2915 [==========>...................] - ETA: 4s - loss: 0.1519 - auc_19: 0.9955 [repeated 2x across cluster]


## Evaluation with test data

The evaluation has been done during the simulation. Following, we show the averaged results over test data.
The result of the simulation includes the results on all rounds, so we retrieve those of the last round.

In [32]:
print('Test data, \t Loss={:.4f}, \t Accuracy={:.4f}'.format(fl_sim.losses_distributed[-1][1], fl_sim.metrics_distributed['TPR*TNR'][-1][1]))

Test data, 	 Loss=0.1276, 	 Accuracy=0.9058


In [33]:
fl_sim

History (loss, distributed):
	round 1: 0.05392354167997837
	round 2: 0.10520271733403205
	round 3: 0.1063789702951908
	round 4: 0.11067049130797386
	round 5: 0.10285662561655044
	round 6: 0.12072182446718216
	round 7: 0.10957779437303543
	round 8: 0.11824006587266922
	round 9: 0.12832544967532158
	round 10: 0.1276228755712509
History (metrics, distributed, evaluate):
{'TPR*TNR': [(1, 0.882634710146813), (2, 0.8970806566225734), (3, 0.8978881872419666), (4, 0.9088566066696251), (5, 0.911663094082676), (6, 0.9160532709168877), (7, 0.9141170593754905), (8, 0.9202585911239172), (9, 0.9082675729549773), (10, 0.9058414615466825)]}